# **PROJET PYTHON POUR LA DATA SCIENCE : Machine Learning vs Black-Scholes**
##        ***Les algorithmes de machine learning prédiraient-ils mieux le cours des actions financières que le célèbre modèle économétrique Black-Scholes?***

## **Auteurs:** 
### **<center> NGUEMOGNE Sandra, Rougier Antoine</center>**

####     L'objectif de ce notebook est de présenter le projet que nous avons effectué dans le cadre de l'unité d'enseignement Python pour la data science dispensée à l'ENSAE. Ce projet a été élaboré de manière libre et comporte, comme attendu, un jeu de données récupéré et traité, une partie visualisation et une partie modélisation.

# **Plan du travail**

## **A) Problématique**
## **B) Installations préalables et importation des modules**
## **I. Récupération, visualisation et traitement de données**
## **II. Analyses descriptives et représentations graphiques**
## **III. Modélisation**
### **III.1 Algorithmes de Machine Learning**
#### **a) Regression linéaire**
#### **b) Modèle SVM** 
### **III.2 Modèle Black-Scholes**
## **C) Ce qu'on retient**

## **A) Problématique**
### Ce projet s'articule autour de la problématique suivante :

#### ***<center>Lequel des algorithmes de Machine Learning et du modèle économétrique de Black-Scholes prédit-il le mieux le cours de l'action de la compagnie d'Assurance AXA?</center>***

## **B) Installations préalables et importation des modules** 

Avant d'exécuter ce code, veuillez procéder aux installations et importations des modules nécessaires à son bon fonctionnement: exécutez la cellule ci-dessous.

In [1]:
!pip install yfinance
!pip install matplotlib
import yfinance as yahooFinance
import datetime
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_X_y
from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 17.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 20.9 MB/s eta 0:00:00


## **I. Récupération, visualisation et traitement de données** 

Dans cette partie, nous récupérons et traitons les données portant sur le cours de l'action de la compagnie d'Assurance AXA. Nous avons choisi de  les récupérer directement sur le site  de référence: ***Yahoo Finance***. Nous récupérons ainsi les données allant du 1er janvier 2019 au 31 décembre de la meme annnée. Pour cela nous avons eu besoin d'un identifiant communément appélé ****ticker***, celui d'AXA correspondant ainsi à 
**CS.PA**.
Comme vous pourez le voir en excutant les prochaines cellules, notre jeu de données est un ensemble de 255 observations et 7 variables: ***Open, High, Low, Close, Volume, Dividends et Stock Splits.*** Les 2 dernières variables ne nous seront pas d'un intéret particulier, raison pour laquelle nous allons les supprimer en chemin.
Pour une journée donnée: 
+ La varialble ***Open (respectivement Close)*** correspond au prix, au matin (respectivement dans la soirée/en fin de journée) de l'action d'AXA sur le marché boursier.
- La variable ***High (respectivement Low)*** correspond au prix le plus élevé (respectivement le plus bas) qu'a atteind la valeur de l'action dans la journée.
+ La variable ***Volume*** représente le nombre d'actions échangées sur le marché dans la journée.

#### ***Récupération***

In [2]:
start_Date = datetime.datetime(2019, 1, 1) #Précise la date de début
end_Date = datetime.datetime(2019, 12, 31) #Précise la date de fin

GetAXAInfo = yahooFinance.Ticker("CS.PA") #Localise les données du cours de l'action d'AXA
df_init=GetAXAInfo.history(start=start_Date,end=end_Date) #parmi ces données, on récupère celles correspondant aux dates qui nous interessent

#### ***Visualisation et traitement de données*** 

In [3]:
print(len(df_init)) #  affiche le nombre d'observations
print(df_init.columns) # affiche la liste des variables
df_init.head() # Affiche les 5 premières lignes du dataframe pour un apercu fluide

255
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02 00:00:00+01:00,13.595197,13.738303,13.455009,13.731002,8028529,0.0,0.0
2019-01-03 00:00:00+01:00,13.599576,13.836142,13.589354,13.617100,6583413,0.0,0.0
2019-01-04 00:00:00+01:00,13.720778,14.087309,13.666748,14.044960,11675578,0.0,0.0
2019-01-07 00:00:00+01:00,14.078548,14.150101,13.900394,13.982169,7530045,0.0,0.0
2019-01-08 00:00:00+01:00,14.027437,14.218734,13.988010,14.034739,7314687,0.0,0.0


On remarque que les dates peuvent poser un problème dans notre modélisation, car elles sont converties sous le format Unix, et que les heures ne nous intérèssent pas particulière dans notre projet. Nous allons d'abord convertir ces dates dans un format qui ne nous posera pas de problème. De plus, en visualisant les valeurs minimales et maximales des 2 dernièrres colonnes, nous remarquons que la colonne ***Dividends*** n'est pas toujours nulle contrairement à ce qu'on pouvait penser. Tandis que ***stock splits l'est***. Dans la suite de notre projet, nous n'utiliserons plus ces colonnes car elle ne sont pas pertinentes donc nous décidons de les supprimer. 

In [4]:
#Formatage des dates
df_init = df_init.reset_index()
df_init["Date"] = df_init["Date"].dt.strftime('%Y-%m-%d')
df_init = df_init.set_index('Date')
df_init.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,13.595197,13.738303,13.455009,13.731002,8028529,0.0,0.0
2019-01-03,13.599576,13.836142,13.589354,13.617100,6583413,0.0,0.0
2019-01-04,13.720778,14.087309,13.666748,14.044960,11675578,0.0,0.0
2019-01-07,14.078548,14.150101,13.900394,13.982169,7530045,0.0,0.0
2019-01-08,14.027437,14.218734,13.988010,14.034739,7314687,0.0,0.0


In [5]:
#Visualisation des maxima des deux dernières colonnes
print(max(df_init['Stock Splits']))
print(max(df_init['Dividends']))

#Suppression des 2 dernières colonnes 
df_init = df_init.drop(columns =  ["Dividends","Stock Splits"])
df_init.head()

0.0
1.34


,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02,13.595197,13.738303,13.455009,13.731002,8028529
2019-01-03,13.599576,13.836142,13.589354,13.617100,6583413
2019-01-04,13.720778,14.087309,13.666748,14.044960,11675578
2019-01-07,14.078548,14.150101,13.900394,13.982169,7530045
2019-01-08,14.027437,14.218734,13.988010,14.034739,7314687


## **II. Analyses descriptives et représentations graphiques**

Nous allons d'abord regarder le graphique des prix de l'action Axa sur l'année 2019. Puis nous analyserons le volume échangé sur chaque mois, pour voir s'il n'existerait pas une corrélation entre le volume échangé et le prix de l'action. 

In [ ]:
df2 = df_init.copy()
new_df = df2[['Open']]
new_df2 = df2[['Close']]
plt.subplot(211)
plt.plot(new_df)
plt.title("Evolution des prix d'ouverture des actions de AXA")
plt.ylabel("Valeurs")
plt.xlabel("Temps")

plt.subplot(212)
plt.plot(new_df2)
plt.title("Evolution des prix de fermeture des actions de AXA")
plt.ylabel("Valeurs")
plt.xlabel("Temps")

Nous voyons sur les graphiques ci-dessus que les prix croissent à un certains rythme et que qu'il n'y a pas d'incohérence dans nos données pour les prix d'ouverture. Nous n'avons pas de valeur abérante qui attirerait particulièrement l'attention. Nous concluons alors que nous pouvons utiliser cette base de donnée pour faire notre modèle.

Par la suite (prochains grapfiques), nous avons aussi représenté un histogramme qui nous montre le nombre de transaction par mois de l'action AXA pour voir si il y a une évolution. On remarque alors que les volumes échangés pendant cette année 2019 restent relativement similaires pour chaque mois, nous ne voyons pas d'écart significatif au cours de cette période. Cela se confirme par le diagramme circulaire juste en-bas où on remarque que les parts correspondant à chaque mois ont quaisiment la même surface.
 

In [ ]:
new_df = df2[['Volume']]
new_df
new_df = new_df.reset_index()
new_df['Month'] = pd.to_datetime(new_df['Date']).dt.strftime('%B')
df_grouped = new_df.groupby(new_df['Month'])


#for key, item in df_grouped:
#    print(key)
#    print(df_grouped.get_group(key))


sums = df_grouped.sum().reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
sums
sums.plot.bar()
plt.show()

sums.plot.pie(subplots=True)
plt.show()

df2 = df2.drop(columns =  ["Close"])
df2.corr()


En ce qui concerne les corrélations éventuelles entre les variables hors mis la variable ***Close*** qui est la variable que l'on cherche à expliquer dans ce travail, nous voyons une corrélation négative entre le prix d'ouverture et les volumes échangés. Nous pouvions nous attendre à ce résultat en pensant à la loi de l'offre et de la demande pour fixer un prix sur en marché. Donc, pas d'anomalie pour l'instant. Nous pouvons aussi constater que le prix de l'action à l'ouverture est fortement correlé positivement aux valeurs minimale et maximale que le prix atteind dans la journée.

In [ ]:
#Matrice des correlations sans la variable à expliquer
df2 = df2.drop(columns =  ["Close"]) #supression de la variable à expliquer dans la base clonée
df2.corr() #affichage de la matrice des correlations

## **III. Modélisation**

A présent nous passons à la partie modélisation de notre travail. Rappellons que notre variable dépendante (à expliquer) est Close qui représente le prix de l'action en fin de journée.

### **III.1 Algorithmes de Machine Learning**
#### **a) Regression linéaire**

On commence par faire un modèle de prédiction en s'appuyant uniquement sur la valeur des prix d'ouverture de l'action pour prédire le prix de fermeture de la journée. Nous allons utiliser scikit-learn pour cette modélisation. 

In [ ]:
reg = LinearRegression()

X= df2[['Open']]
X = X[1:(len(X))]
p = int((2/3)*len(X))
X_train = X[:p]
X_test = X[p:]
reg = linear_model.LinearRegression()
reg.fit(X_train, Y_train)
print(reg.coef_)

y_pred = reg.predict(X_test)
plt.subplot(121)
plt.plot(y_pred, color='green')
plt.plot(Y_test, color='gray')
plt.title("Prédiction du cours de l'action AXA")

plt.subplot(122)
plt.scatter(X_test, Y_test, color="black")
plt.plot(X_test, y_pred, color="blue", linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

print("Mean squared error: %.2f" % mean_squared_error(Y_test, y_pred))
print("Coefficient of determination: %.2f" % r2_score(Y_test, y_pred))

On remarque ici que les courbes vertes et grises se chevauchent assez régulièrement, ce qui montrent que la régression est possible.

Mais nous voulons voir si nous ne pouvons pas aller plus loin dans cette modélisation en rajoutant une nouvelle variable qui prendrait en compte les prix de fermeture de la veille. Nous allons donc créer cette nouvelle colonne dans notre tableau pour faire la modélisation.

In [ ]:
dec=df_init["Close"].shift(1)
df_init= df_init.assign(Close_veille=dec)
df_init.head()

###### ***Modélisation en ajoutant la variable que l'on vient de créer***

In [ ]:
#On fait alors une régression avec commes variables explicatives les prix d'ouverture et de fermeture la veille
X = df_init[['Open', 'Close_veille']]
Y = df_init[['Close']]
X = X[1:len(X)]
Y = Y[1:len(Y)]



#On définit l'échantillon des tests, pour cela il nous faut une proportion de prélèvement des données pour les tests; on la note p
p = int((2/3)*len(X))
X_train = X[:p]
Y_train = Y[:p]
X_test = X[p:]
Y_test = Y[p:]

#On passe désormais à l'entrainement et la prédiction 
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
print(regr.coef_)

y_pred = regr.predict(X_test)
plt.plot(y_pred, color='green')
plt.plot(Y_test, color='gray')
plt.title("Prédiction du cours de l'action AXA")

print("Mean squared error: %.2f" % mean_squared_error(Y_test, y_pred))
print("Coefficient of determination: %.2f" % r2_score(Y_test, y_pred))